In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import pprint
import os

In [2]:
# stations data path
file_path = '../Resources/stationslist_with_Routetrip.csv'

# Read CSV and convert to stations_df
stations_df=pd.read_csv(file_path)

# display df
stations_df

,latitude,longitude,stations_Route_trip,stationCode
0,-29.063612,114.997161,4,AN001
1,-34.270827,118.268523,2,AM001
2,-32.125480,118.004060,0,SH002
3,-30.338049,115.539491,4,BA
4,-33.796200,116.063980,2,BP001
...,...,...,...,...
219,-34.225653,116.017767,2,MA003
220,-31.915622,119.256055,3,YS
221,-31.835878,116.921077,0,YE001
222,-28.337630,114.989800,4,YU001


In [3]:
# rain data path
file_path = '../Resources/raindata_with_perdiction.csv'

# Read CSV and convert to rain_df
rain_df=pd.read_csv(file_path)

# Drop NA value from dataframe
rain_df.dropna(subset=['Year', 'Month', 'Day','stationCode','stationName','relativeHumidity','airTemperature_avg','wind_avg_speed','Actual_Rain','Predicted_Rain'], inplace=True)

# Convert date into number
rain_df['Date'] = pd.to_datetime(rain_df[['Year', 'Month', 'Day']], errors='coerce')

# X = rain_df.drop(["rainfall","stationName"],axis=1).values
rain_df= rain_df.drop(["Year","Month","Day"],axis=1)

rain_df

,stationCode,stationName,relativeHumidity,airTemperature_avg,wind_avg_speed,Actual_Rain,Predicted_Rain,Date
0,AN001,Allanooka,66.1,21.0,17.93,0.0,2.0,2023-01-01
1,AN001,Allanooka,52.2,25.2,16.49,0.0,0.0,2023-01-02
2,AN001,Allanooka,19.1,32.2,24.77,0.0,0.0,2023-01-03
3,AN001,Allanooka,22.9,32.1,21.67,0.0,0.0,2023-01-04
4,AN001,Allanooka,17.8,33.3,18.12,2.8,0.0,2023-01-05
...,...,...,...,...,...,...,...,...
47159,YU002,Yuna NE,74.2,14.2,5.72,0.0,0.0,2023-08-09
47160,YU002,Yuna NE,72.1,14.8,5.71,8.6,3.0,2023-08-10
47161,YU002,Yuna NE,72.8,16.8,6.62,0.0,0.0,2023-08-11
47162,YU002,Yuna NE,75.7,15.9,7.51,0.0,0.0,2023-08-12


In [ ]:
# Initialize a dictionary to store dictionaries for each station
all_stations = {}

# Iterate through each station
for station in stations_df['stationCode']:
    # Filter rows based on the 'station' column for the rain data
    filtered_row = rain_df[rain_df['stationCode'] == station]
    
    # Initialize empty arrays to store all rain data for each station for different dates
    rain_array = []
    prain_array = []
    date_array = []

    # Iterate through the DataFrame and populate the rain amount arrays
    for i in range(len(filtered_row)):
        rain = filtered_row.iloc[i]['Actual_Rain']
        prain = filtered_row.iloc[i]['Predicted_Rain']
        date = (pd.to_datetime(filtered_row.iloc[i]['Date']).date()).strftime("%Y-%m-%d")

        rain_array.append(rain)
        prain_array.append(prain)
        date_array.append(date)

    # Create a dictionary with the desired structure for the current rain station
    station_dict = {
        "stationCode": station,
        "lat": stations_df[stations_df['stationCode'] == station]['latitude'].values[0],
        "lon": stations_df[stations_df['stationCode'] == station]['longitude'].values[0],
        "Routetrip": stations_df[stations_df['stationCode'] == station]['stations_Route_trip'].values[0],
        "Date": date_array,
        "Rain": rain_array,
        "PerdictRain": prain_array
    }

    # Update the all_stations dictionary with the station_dict
    all_stations[station] = station_dict

# Display the result
# all_stations
    

In [ ]:
# Function to convert NumPy integers to regular Python integers
def convert_numpy_int(item):
    if isinstance(item, np.int64):
        return int(item)
    raise TypeError(f"Type {type(item)} not serializable")

# Convert dictionary to JSON string, using the custom converter
json_data = json.dumps(all_stations, indent=2, default=convert_numpy_int)

# Define the JavaScript variable assignment
js_content = f"var alldata = {json_data};"

# Write the JavaScript content to a file
with open('../Map/static/data/data.js', 'w') as js_file:
    js_file.write(js_content)